# Pump it Up: Data Mining the Water Table

### Can you predict which water pumps are faulty?
Using data from Taarifa and the Tanzanian Ministry of Water, can you predict which pumps are functional, which need some repairs, and which don't work at all? This is an intermediate-level practice competition. Predict one of these three classes based on a number of variables about what kind of pump is operating, when it was installed, and how it is managed. A smart understanding of which waterpoints will fail can improve maintenance operations and ensure that clean, potable water is available to communities across Tanzania.

Competition:
https://www.drivendata.org/competitions/7/

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns

import sklearn as sk
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, cross_val_predict, cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

import xgboost as xgb

In [ ]:
# Download the data and load with pandas
TRAIN_DATA_URL = 'https://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv'
TRAIN_LABELS_URL = 'https://s3.amazonaws.com/drivendata/data/7/public/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv'
TEST_DATA_URL = 'https://s3.amazonaws.com/drivendata/data/7/public/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv'

train_values = pd.read_csv(TRAIN_DATA_URL,
                           index_col='id')
train_targets = pd.read_csv(TRAIN_LABELS_URL,
                            index_col='id')
test_values = pd.read_csv(TEST_DATA_URL,
                          index_col='id')

In [ ]:
# merge training values and labels as a copy
train = pd.merge(train_values, train_targets, left_index=True, right_index=True).copy()
test = test_values.copy()

def status_group_mapper(status_group: str):
    if status_group == 'functional':
        return 2
    elif status_group == 'non functional':
        return 0
    else:
        return 1

# map status_group label to numeric class (0: non-func, 1: repair, 2: func)
train['status_group'] = train['status_group'].apply(status_group_mapper)
train.head()

In [ ]:
# all object-columns to lower-case
def dataset_string_to_lowercase(df: pd.DataFrame):
    for cname in df.columns:
        if df[cname].dtype == object:
            df[cname] = df[cname].str.lower()

dataset_string_to_lowercase(train)
dataset_string_to_lowercase(test)

### Explore the dataset

- amount_tsh - Total static head (amount water available to waterpoint)
- date_recorded - The date the row was entered
- funder - Who funded the well
- gps_height - Altitude of the well
- installer - Organization that installed the well
- longitude - GPS coordinate
- latitude - GPS coordinate
- wpt_name - Name of the waterpoint if there is one
- num_private -
- basin - Geographic water basin
- subvillage - Geographic location
- region - Geographic location
- region_code - Geographic location (coded)
- district_code - Geographic location (coded)
- lga - Geographic location
- ward - Geographic location
- population - Population around the well
- public_meeting - True/False
- recorded_by - Group entering this row of data
- scheme_management - Who operates the waterpoint
- scheme_name - Who operates the waterpoint
- permit - If the waterpoint is permitted
- construction_year - Year the waterpoint was constructed
- extraction_type - The kind of extraction the waterpoint uses
- extraction_type_group - The kind of extraction the waterpoint uses
- extraction_type_class - The kind of extraction the waterpoint uses
- management - How the waterpoint is managed
- management_group - How the waterpoint is managed
- payment - What the water costs
- payment_type - What the water costs
- water_quality - The quality of the water
- quality_group - The quality of the water
- quantity - The quantity of water
- quantity_group - The quantity of water
- source - The source of the water
- source_type - The source of the water
- source_class - The source of the water
- waterpoint_type - The kind of waterpoint
- waterpoint_type_group - The kind of waterpoint

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
def matrix_plot(cols):
    sns.set(style='whitegrid',context='notebook')
    sns.pairplot(train[cols],size=2.5)
    plt.show()
    
matrix_plot(['amount_tsh','num_private','population','status_group'])
matrix_plot(['gps_height','latitude', 'longitude','status_group'])

In [ ]:
# check for missing values
train.isna().sum()

In [ ]:
def check_redundant(col1, col2):
    result = np.where(train[col1] != train[col2])[0]
    print('{} != {}: {}/{}'.format(col1, col2, result.size, len(train)))

# check redundancies
check_redundant('quantity', 'quantity_group')
check_redundant('waterpoint_type', 'waterpoint_type_group')
check_redundant('water_quality', 'quality_group')
check_redundant('management', 'management_group')
check_redundant('payment', 'payment_type') # different naming, but is identical
check_redundant('source', 'source_type')
check_redundant('source', 'source_class')
check_redundant('extraction_type', 'extraction_type_group')
check_redundant('extraction_type', 'extraction_type_class')
check_redundant('funder', 'installer')

### Data preparation

#### quantity, quantity_group

In [ ]:
# TODO remove or merge redundant columns
train = train.drop(columns=['quantity_group'])
test = test.drop(columns=['quantity_group'])

#### population

In [ ]:
train.loc[train['population']>0, 'population'].plot(kind='hist', title='TRAIN', bins=50)
plt.show()
test.loc[test['population']>0, 'population'].plot(kind='hist', title='TEST', bins=50)
plt.show()

# TODO: binning ? (p<10, 10<=p<100, 100<=p<500, 500<=p<1000, 1000<=p, ...)

#### funder, installer

In [ ]:
# replace 0 (zero) / NAs in columns other/unknown
train = train.drop(columns=['funder'])
train = train.replace({'installer':'0'}, 'unknown')
train = train.replace({'installer': np.nan}, 'unknown')

test = test.drop(columns=['funder'])
test = test.replace({'installer':'0'}, 'unknown')
test = test.replace({'installer': np.nan}, 'unknown')

In [ ]:
def replace_mapper(value: str, contains: str, label: str):
    if contains in value:
        return label
    return value

train['installer'] = train['installer'].apply(lambda v : replace_mapper(v, 'gov', 'government'))
train['installer'] = train['installer'].apply(lambda v : replace_mapper(v, 'comm', 'community'))
train['installer'] = train['installer'].apply(lambda v : replace_mapper(v, 'danid', 'danida'))

test['installer'] = test['installer'].apply(lambda v : replace_mapper(v, 'gov', 'government'))
test['installer'] = test['installer'].apply(lambda v : replace_mapper(v, 'comm', 'community'))
test['installer'] = test['installer'].apply(lambda v : replace_mapper(v, 'danid', 'danida'))

In [ ]:
installer = train['installer'].value_counts()
installer_few = train['installer'].isin(installer.index[installer < 100])
train.loc[installer_few, 'installer'] = 'other'
train['installer'].value_counts()

In [ ]:
installer_values = train['installer'].unique()

test['installer'] = test['installer'].apply(lambda v: 'other' if v not in installer_values else v)

#### waterpoint, waterpoint_type_group

In [ ]:
print(train['waterpoint_type'].value_counts(dropna=False))
print(train['waterpoint_type_group'].value_counts(dropna=False))

In [ ]:
# make dam (only 7 instances) to others, and drop waterpoint_type_group
train = train.replace({'waterpoint_type':'dam'}, 'other')
train = train.drop(columns=['waterpoint_type_group'])

test = test.replace({'waterpoint_type':'dam'}, 'other')
test = test.drop(columns=['waterpoint_type_group'])

#### construction_year, date_recorded, well_age

In [ ]:
# trim the record date to the year
train['date_recorded'] = train['date_recorded'].apply(lambda v : float(v[:4]))
test['date_recorded'] = test['date_recorded'].apply(lambda v : float(v[:4]))

In [ ]:
# fill construction_year==0 with the min value (I expect that the year is unknown because it's a while ago)
train = train.replace({'construction_year':0}, np.nan)
test = test.replace({'construction_year':0}, np.nan)
min_construction_year = int(train['construction_year'].min())
train = train.replace({'construction_year':np.nan}, min_construction_year)
test = test.replace({'construction_year':np.nan}, min_construction_year)

# create a new column 'well_age'
train['well_age'] = train['date_recorded'] - train['construction_year']
test['well_age'] = test['date_recorded'] - test['construction_year']

In [ ]:
train.loc[train['well_age'] < 0, ['date_recorded', 'construction_year', 'well_age', 'status_group']]

date_recorded < construction_year seams to be an error. Since date_recorded is always 2004, this looks like an systematic error. Could be a typo and 2014 instead of 2014.

Here, we simply make a value zero.

In [ ]:
train.loc[train['well_age'] < 0, 'well_age'] = 0
test.loc[test['well_age'] < 0, 'well_age'] = 0

#### extraction_type, extraction_type_group, extraction_type_class

In [ ]:
print(train['extraction_type'].value_counts())
print(train['extraction_type_group'].value_counts())
print(train['extraction_type_class'].value_counts())

In [ ]:
# we only keep extraction_type_class, since all are quite similar and this one has the most understandable groups
train = train.drop(columns=['extraction_type', 'extraction_type_class'])
test = test.drop(columns=['extraction_type', 'extraction_type_class'])

train = train.replace({'extraction_type_group':'india mark ii'}, 'india mark')
train = train.replace({'extraction_type_group':'india mark iii'}, 'india mark')
test = test.replace({'extraction_type_group':'india mark ii'}, 'india mark')
test = test.replace({'extraction_type_group':'india mark iii'}, 'india mark')

train['extraction_type_group'].value_counts()

In [ ]:
selected = train[train['extraction_type_group'] == 'wind-powered']
cross_table = pd.crosstab(index=selected['extraction_type_group'], columns=train['status_group'])
cross_table
cross_table.plot(kind='bar', stacked=True)
plt.show()

#### latitude, longitude, gps_height
- there are many zero (0 or -2e-8) which is not withing Tanzania (lat: (-11, -1), lng: (30, 40))
- height extremes in Tanzania are 0 to 5,895m. Minus values might be underground. Many empty 0 values.
- use basin to estimate the apprx. location

In [ ]:
# ensure all unknown values are zero (0):
train = train.replace({'latitude':-2.000000e-08}, 0)
test = test.replace({'latitude':-2.000000e-08}, 0)

# bounds of min/max latitude/longitude/height for Tanzania using basin
train_bound = train[(train['latitude']!=0)&(train['longitude']!=0)&(train['gps_height']!=0)]
train_median_geo = train_bound.groupby(['basin',])['latitude','longitude','gps_height'].median()
train_median_geo

In [ ]:
train.loc[train['gps_height']==0, 'gps_height'] = train['basin'].apply(lambda x : train_median_geo.at[x,'gps_height'])
train.loc[train['longitude']==0, 'longitude'] = train['basin'].apply(lambda x : train_median_geo.at[x,'longitude'])
train.loc[train['latitude']==0, 'latitude'] = train['basin'].apply(lambda x : train_median_geo.at[x,'latitude'])

test.loc[test['gps_height']==0, 'gps_height'] = test['basin'].apply(lambda x : train_median_geo.at[x,'gps_height'])
test.loc[test['longitude']==0, 'longitude'] = test['basin'].apply(lambda x : train_median_geo.at[x,'longitude'])
test.loc[test['latitude']==0, 'latitude'] = test['basin'].apply(lambda x : train_median_geo.at[x,'latitude'])

#### management

In [ ]:
train[['scheme_management', 'scheme_name', 'management', 'management_group']].head(n=20)

In [ ]:
# print(train['scheme_management'].value_counts()) # almost same as management, but with NaN values
# print(train['scheme_name'].value_counts()) # 2576 different values!
print(train['management'].value_counts())
print(train['management_group'].value_counts())

In [ ]:
train = train.replace({'management':'unknown'}, 'other')
train = train.replace({'management': 'other - school'}, 'other')
train = train.replace({'management': 'trust'}, 'other')
train = train.replace({'management_group':'unknown'}, 'other')

test = test.replace({'management':'unknown'}, 'other')
test = test.replace({'management': 'other - school'}, 'other')
test = test.replace({'management': 'trust'}, 'other')
test = test.replace({'management_group':'unknown'}, 'other')

#### region / district

In [ ]:
train[['region', 'region_code', 'district_code']].head(n=10)

region, region_code and district seem to be very similar. We will use region and binarize it later.

#### basin

In [ ]:
print(train['basin'].value_counts())

#### source

In [ ]:
print(train['source'].value_counts())
print(train['source_type'].value_counts())
print(train['source_class'].value_counts())

In [ ]:
train = train.replace({'source':'unknown'}, 'other')
test = test.replace({'source':'unknown'}, 'other')

#### payment

In [ ]:
print(train['payment'].value_counts())
print(train['payment_type'].value_counts())

## Visualize cleaned data

Some more visualization after the data has been cleaned up

In [ ]:
def crosstab_diagram(data: pd.DataFrame, index_col: str, figsize=None):
    cross_table = pd.crosstab(index=data[index_col], columns=data['status_group'])
    cross_table.plot(kind='bar', stacked=True, figsize=figsize)
    plt.show()

In [ ]:
crosstab_diagram(train, 'quantity')

In [ ]:
crosstab_diagram(train, 'waterpoint_type')

In [ ]:
crosstab_diagram(train, 'extraction_type_group')

In [ ]:
crosstab_diagram(train, 'region')

In [ ]:
crosstab_diagram(train, 'management')

In [ ]:
crosstab_diagram(train, 'management_group')

In [ ]:
crosstab_diagram(train, 'source')

In [ ]:
crosstab_diagram(train, 'source_class') # does not seam to be helpful

In [ ]:
crosstab_diagram(train, 'payment')

In [ ]:
crosstab_diagram(train, 'installer', figsize=(16,4))

In [ ]:
crosstab_diagram(train, 'basin')

In [ ]:
crosstab_diagram(train, 'well_age')

In [ ]:
crosstab_diagram(train, 'construction_year')

## Attributes that we are interested in

In [ ]:
numeric_cols = ['gps_height', 'latitude', 'longitude', 'well_age', 'construction_year', 'population']
ordered_cat_cols = []
unordered_cat_cols = ['quantity', 'waterpoint_type', 'extraction_type_group', 'region',\
                      'management', 'source',\
                      'payment', 'installer', 'basin'] # source_class, management_group
all_cols = numeric_cols + ordered_cat_cols + unordered_cat_cols

## Save preprocessed data

In [ ]:
train[all_cols].to_csv('data/train_clean.csv')
test[all_cols].to_csv('data/test_clean.csv')

## Encode categorical attributes
- LabelEncoder (ordered)
- LabelBinarizer (unordered)

In [ ]:
def encode_categories(column: pd.core.series.Series):
    encoder = LabelBinarizer()
    return encoder.fit_transform(column)

In [ ]:
# for column_name in unordered_cat_cols:
    # train[column_name] = encode_categories(train[column_name])

In [ ]:
# encode_categories(train['quantity'])
# train['quantity']

In [ ]:
training = pd.get_dummies(train[all_cols], columns=unordered_cat_cols)
training

In [ ]:
testing = pd.get_dummies(test[all_cols], columns=unordered_cat_cols)
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

dfTest[['A','B','C']] = scale.fit_transform(dfTest[['A','B','C']].as_matrix()) ??? # TODO

In [ ]:


normalized_df=(df-df.mean())/df.std()

training = pd.ge

In [ ]:
training.columns

In [ ]:
testing.columns

In [ ]:
train_data = training.as_matrix()
train_labels = train['status_group'].as_matrix()
test_data = testing.as_matrix()

## Model selection

#### Random Forest

In [ ]:
model = RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=16, max_features='log2',
            max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0, warm_start=False,
            random_state=42)
model

#### XGBoost

In [ ]:
model = xgb.XGBClassifier(max_depth=10, n_estimators=500, learning_rate=0.1, reg_alpha=0, reg_lambda=1.0,
                          random_state=42)
model

#### Simple train/test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=0)
#X_train, y_train = (train_data, train_labels)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
pred_test = model.predict(X_test)
plt.hist(pred_test)

In [ ]:
accuracy_score(y_true=y_test, y_pred=pred_test)

In [ ]:
pred_train = model.predict(X_train)
accuracy_score(y_true=y_train, y_pred=pred_train)

In [ ]:
confmat = confusion_matrix(y_true=y_train, y_pred=pred_train)
confmat

In [ ]:
print(classification_report(y_true=y_test, y_pred=prediction))

In [ ]:
xgb.plot_importance(model)

#### Cross-validation

In [ ]:
model_scores = cross_val_score(model, train_data, train_labels, cv=5)

In [ ]:
print('Scores: {}'.format(model_scores))
print('Mean: {}'.format(model_scores.mean()))
print('Std: {}'.format(model_scores.std()))

## Predictions for submission

In [ ]:
prediction = model.predict(test_data)
plt.hist(prediction)

In [ ]:
def save_submission(predictions, test):
    data = {'id': test.index, 'status_group': predictions}

    submit = pd.DataFrame(data=data)

    vals_to_replace = {0:'non functional',
                       1:'functional needs repair',
                       2:'functional'}

    submit['status_group'] = submit['status_group'].replace(vals_to_replace)        

    submit.to_csv('pump_predictions.csv', index=False)

In [ ]:
save_submission(prediction, test)